In [2]:
import requests
import json
import pandas as pd
import pymysql

In [ ]:
#genre information crawling (for genre table)

genre_url="https://api.themoviedb.org/3/genre/movie/list?api_key=36927ad1d2817ff0fa31947e47d186fd&language=ko-KR"
response = requests.get(genre_url)
json_object = response.content
genre_data = json.loads(json_object)
genre_data_df = pd.DataFrame(genre_data['genres'])
print(genre_data_df)

#genre_data_df.to_csv("genre.csv")

In [ ]:
# genre table에 영화 장르정보 삽입
#!pip install pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

cursor.execute('DELETE FROM genre')
for id, name in genre_data_df.values:
    # print("{0}/{1}".format(id, name))
    sql= "INSERT INTO genre VALUES(%s, %s)"
    cursor.execute(sql, (id, name))

conn.commit()
cursor.close()
conn.close()


In [3]:
# 2021년도 영화(2021-11-25일까지)크롤링 (for movie table)
movies_2021 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2021-01-01&vote_count.gte=100&primary_release_date.lte=2021-11-25&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2021.append(movies['results'])


movies = []
for row in movies_2021:
    for r in row:
       movies.append(r)
movies_2021_df=pd.DataFrame(movies)
movies_2021_modified = movies_2021_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2021_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

# movies_2021_modified.info()   #272 entries
# movies_2021_modified.to_csv("movies_2021.csv")

In [10]:
#2021년도 영화(2021-11-25일까지) movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

cursor.execute('DELETE FROM movie')

for row in movies_2021_modified.values:
     cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [9]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2021년도 movie_id, genre_id삽입

movie_genre_ids_2021_df=movies_2021_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()
cursor.execute('DELETE FROM movie_genre')

for movie_genre_ids in movie_genre_ids_2021_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        # print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [ ]:
# 2020년도 영화 크롤링 (for movie table)
movies_2020 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2020-01-01&vote_count.gte=100&primary_release_date.lte=2020-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2020.append(movies['results'])


movies = []
for row in movies_2020:
    for r in row:
       movies.append(r)
movies_2020_df=pd.DataFrame(movies)
movies_2020_modified = movies_2020_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2020_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#movies_2020_modified.info()   #458 entries

In [14]:
#2020년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2020_modified.values:
     cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [10]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2020년도 movie_id, genre_id삽입

movie_genre_ids_2020_df=movies_2020_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2020_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        # print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [7]:
# 2019년도 영화 크롤링 (for movie table)
movies_2019 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2019-01-01&vote_count.gte=100&primary_release_date.lte=2019-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2019.append(movies['results'])


movies = []
for row in movies_2019:
    for r in row:
       movies.append(r)
movies_2019_df=pd.DataFrame(movies)
movies_2019_modified = movies_2019_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2019_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#movies_2019_modified.info() # 458 entries
#movies_2019_modified.tail()


In [ ]:
#2019년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2019_modified.values:
     cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [12]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2019년도 movie_id, genre_id삽입

movie_genre_ids_2019_df=movies_2019_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2019_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        # print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [19]:
# 2018년도 영화 크롤링 (for movie table)
movies_2018 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2018-01-01&vote_count.gte=100&primary_release_date.lte=2018-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2018.append(movies['results'])


movies = []
for row in movies_2018:
    for r in row:
       movies.append(r)
movies_2018_df=pd.DataFrame(movies)
movies_2018_modified = movies_2018_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2018_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#movies_2018_modified.info() # 707 entries
#movies_2018_modified.tail()

In [ ]:
#2018년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2018_modified.values:
    cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [ ]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2018년도 movie_id, genre_id삽입

movie_genre_ids_2018_df=movies_2018_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2018_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        #print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [21]:
# 2017년도 영화 크롤링 (for movie table)
movies_2017 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2017-01-01&vote_count.gte=100&primary_release_date.lte=2017-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2017.append(movies['results'])


movies = []
for row in movies_2017:
    for r in row:
       movies.append(r)
movies_2017_df=pd.DataFrame(movies)
movies_2017_modified = movies_2017_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2017_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#movies_2017_modified.info() # 719 entries
#movies_2017_modified.tail()

In [25]:
#2017년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2017_modified.values:
    cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [22]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2017년도 movie_id, genre_id삽입

movie_genre_ids_2017_df=movies_2017_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2017_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        #print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [23]:
# 2016년도 영화 크롤링 (for movie table)
movies_2016 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2016-01-01&vote_count.gte=100&primary_release_date.lte=2016-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2016.append(movies['results'])


movies = []
for row in movies_2016:
    for r in row:
       movies.append(r)
movies_2016_df=pd.DataFrame(movies)
movies_2016_modified = movies_2016_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2016_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#movies_2016_modified.info() # 673 entries
#movies_2016_modified.tail()

In [28]:
#2016년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2016_modified.values:
    cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [24]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2016년도 movie_id, genre_id삽입

movie_genre_ids_2016_df=movies_2016_modified[['movie_id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2016_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        #print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [25]:
# 2015년도 영화 크롤링 (for movie table)
movies_2015 = []
for page in range(1,50):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2015-01-01&vote_count.gte=100&primary_release_date.lte=2015-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2015.append(movies['results'])


movies = []
for row in movies_2015:
    for r in row:
       movies.append(r)
movies_2015_df=pd.DataFrame(movies)
movies_2015_modified = movies_2015_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2015_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

# movies_2015_modified.info() # 626 entries
# movies_2015_modified.tail()

In [30]:
#2015년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()
for row in movies_2015_modified.values:
    cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [26]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2015년도 movie_id, genre_id삽입

movie_genre_ids_2015_df=movies_2015_modified[['movie_id','genre_ids']]
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2015_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        #print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()

In [27]:
# 2010~2014년도 영화 크롤링 (for movie table)
movies_2010_2014 = []
for page in range(1,250):
    url="https://api.themoviedb.org/3/discover/movie/?api_key={0}&language=ko-KR&primary_release_date.gte=2010-01-01&vote_count.gte=100&primary_release_date.lte=2014-12-31&sort_by=primary_release_date.asc&page={1}".format('5ecfbcb3280f5e9f7898c3ca00acca81',page) 
    response = requests.get(url)
    movies= json.loads(response.content)
    movies_2010_2014.append(movies['results'])


movies = []
for row in movies_2010_2014:
    for r in row:
       movies.append(r)
movies_2010_2014_df=pd.DataFrame(movies)
movies_2010_2014_modified = movies_2010_2014_df.drop(["adult", "backdrop_path","video"], axis=1)  # 필요없는 칼럼제거 
movies_2010_2014_modified.rename(columns={'id':'movie_id', 'original_title':'subtitle', 'poster_path': 'posterpath'}, inplace=True)   # 컬럼이름을 DB컬럼이름과 통일. 

#ovies_2010_2014_modified.info() # 2636 entries
#movies_2010_2014_modified.tail()

In [4]:
#2010-2014년도 영화정보 movie table (mysql)에 삽입
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
                       
cursor = conn.cursor()

for row in movies_2010_2014_modified.values:
    cursor.execute('INSERT INTO movie (movie_id, original_language, subtitle, overview, popularity, posterpath, release_date, title, vote_average, vote_count) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s)', list(row[[1,2,3,4,5,6,7,8,9,10]]))
conn.commit()
cursor.close()
conn.close()

In [28]:
# movie_genre table (genre table & movie table을 위한 연결테이블) 에 2010-2014년도 movie_id, genre_id삽입

movie_genre_ids_2010_2014_df=movies_2010_2014_modified[['movie_id','genre_ids']]
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()

for movie_genre_ids in movie_genre_ids_2010_2014_df.values:
    #print(movie_genre_ids[1])
    #break
    movie_id = movie_genre_ids[0]
    for genre_id in movie_genre_ids[1]:
        sql= "INSERT INTO movie_genre VALUES (%s, %s)"
        cursor.execute(sql, (movie_id, genre_id))
        #print((movie_id, genre_id))

conn.commit()
cursor.close()
conn.close()